In [15]:
import igl
import numpy as np
import meshplot as mp

In [16]:
from collections import deque

def build_geodesic_graph(V, F):
    n = V.shape[0]
    adj = [[] for _ in range(n)]
    # triangle–triangle adjacency
    TT, _ = igl.triangle_triangle_adjacency(F)
    for f_idx in range(F.shape[0]):
        tri = F[f_idx]
        for loc in range(3):
            v = int(tri[loc])
            w1, w2 = int(tri[(loc+1)%3]), int(tri[(loc+2)%3])
            # primal edges
            for w in (w1, w2):
                d = np.linalg.norm(V[v] - V[w])
                adj[v].append((w, d))
                adj[w].append((v, d))
            # dual edges
            f2 = int(TT[f_idx, loc])
            if f2 < 0:
                continue
            edge_vs = {w1, w2}
            vp = next(int(x) for x in F[f2] if int(x) not in edge_vs)
            # compute “geodesic” dual‐edge length via corner angles
            w = w1
            vec_vw = V[v] - V[w]
            vec_uw = V[w2] - V[w]
            vec_vpw = V[vp] - V[w]
            nv, nu, npv = np.linalg.norm(vec_vw), np.linalg.norm(vec_uw), np.linalg.norm(vec_vpw)
            alfa = np.arccos(np.clip(np.dot(vec_vw, vec_uw)/(nv*nu), -1,1))
            beta = np.arccos(np.clip(np.dot(vec_vpw, vec_uw)/(npv*nu), -1,1))
            dual_len = np.sqrt(nv**2 + npv**2 - 2*nv*npv*np.cos(alfa+beta))
            adj[v].append((vp, dual_len))
            adj[vp].append((v, dual_len))
    return adj

def geodesic_distances(adj, sources):
    n = len(adj)
    dist = np.full(n, np.inf)
    prev = [None]*n
    Q = deque()
    for s in sources:
        dist[s] = 0.0
        Q.append(s)
    while Q:
        u = Q.popleft()
        du = dist[u]
        for v, w in adj[u]:
            alt = du + w
            if alt < dist[v]:
                dist[v] = alt
                prev[v] = u
                # SLF–LLL
                if Q and alt < dist[Q[0]]:
                    Q.appendleft(v)
                else:
                    Q.append(v)
    return dist, prev


In [17]:
# ——— Usage with meshplot ———
# 1. Load your mesh
V, F = igl.read_triangle_mesh("./data/bunny.off")
mp.plot(V, F, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

In [29]:
# 2) Compute geodesics
adj = build_geodesic_graph(V, F)

src = 0

dist, _ = geodesic_distances(adj, [src])

# 3a) Simple Viridis (default)
plotter = mp.plot(V, F, c=dist, shading={"wireframe": False})

# 5) Overlay the source as a red point
plotter.add_points(
    V[[src], :],              # shape (1,3)
    c=[[1.0, 0.0, 0.0]],       # red
)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Invalid color array given! Supported are numpy arrays. <class 'list'>


1